In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from skimage import io
import numpy as np
from image_helper import correct_grayscale
from IPython.display import display, clear_output
from particle_helper import find_centroid, create_particle
from records import RADIUS, TERMINAL_VELOCITIES
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from plot_helper import plot_shadow, compare_2d
import json
with open('particle_configurations.json') as f:
    configurations = json.load(f)
import time
from optimizer import optimize_rotation_angle

mpl.rc('figure', figsize=(10, 6))
mpl.rc('image', cmap='gray')

# Load the .tif image, and then invert it so that the particle appears white
frames = np.invert(io.imread("1Steel&1Cu_CopperUp_27_6fps_in crop.tif"))

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


In [2]:
corrected_images = correct_grayscale(frames)

In [3]:
%matplotlib notebook
config_name = "dimer-st-cu"
p = create_particle(config_name)

In [4]:
# Initial guess for the rotation angle
initial_theta = 0
# List to store the optimal theta values
optimal_thetas = []

# Record the start time
start_time = time.time()
# Loop through frames
for fr in range(10, 325):
    # Find the optimal rotation angle
    optimal_theta = optimize_rotation_angle(p, corrected_images[fr], initial_theta, search_range=10)
    # Use the found optimal rotation angle as the initial guess for the next frame
    initial_theta = optimal_theta
    # Append the optimal theta to the list
    optimal_thetas.append(optimal_theta)
    print(f"Frame:{fr}, Optimal Theta: {optimal_theta}")

# Record the end time
end_time = time.time()
# Compute and print the elapsed time
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Frame:10, Optimal Theta: 0
Frame:11, Optimal Theta: 10
Frame:12, Optimal Theta: 19
Frame:13, Optimal Theta: 9
Frame:14, Optimal Theta: 9
Frame:15, Optimal Theta: 17
Frame:16, Optimal Theta: 20
Frame:17, Optimal Theta: 30
Frame:18, Optimal Theta: 36
Frame:19, Optimal Theta: 45
Frame:20, Optimal Theta: 55
Frame:21, Optimal Theta: 62
Frame:22, Optimal Theta: 70
Frame:23, Optimal Theta: 75
Frame:24, Optimal Theta: 83
Frame:25, Optimal Theta: 91
Frame:26, Optimal Theta: 98
Frame:27, Optimal Theta: 98
Frame:28, Optimal Theta: 98
Frame:29, Optimal Theta: 98
Frame:30, Optimal Theta: 97
Frame:31, Optimal Theta: 97
Frame:32, Optimal Theta: 97
Frame:33, Optimal Theta: 97
Frame:34, Optimal Theta: 97
Frame:35, Optimal Theta: 96
Frame:36, Optimal Theta: 96
Frame:37, Optimal Theta: 96
Frame:38, Optimal Theta: 95
Frame:39, Optimal Theta: 95
Frame:40, Optimal Theta: 95
Frame:41, Optimal Theta: 94
Frame:42, Optimal Theta: 96
Frame:43, Optimal Theta: 101
Frame:44, Optimal Theta: 101
Frame:45, Optimal The

In [5]:
fps = 6  # frame rate of the .tif images
name_light = 'st'  # name of the lightest particle
v_light_term = TERMINAL_VELOCITIES[name_light] # terminal velocity of the lightest particle
tau = RADIUS/v_light_term  # timescale of the sedimentation process
# normalize the times by the timescale tau
scaled_time = np.arange(len(optimal_thetas)) / (fps * tau)

# Convert the optimal theta values to radians
optimal_thetas_rad = np.radians(optimal_thetas)

# Set the style of the visualization
sns.set_style("whitegrid")
plt.figure(figsize=(12, 6))  # Set the size of the figure

# Plotting the optimal theta values
plt.plot(scaled_time, optimal_thetas_rad, marker='o', linestyle='-', color=sns.color_palette("viridis", n_colors=3)[0])
plt.title(r'Optimal $\theta$ over Time', fontsize=16)
plt.xlabel(r't /$\tau$', fontsize=14)
plt.ylabel(r'$\theta$', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True, linestyle='dotted', alpha=0.7)

# Set the y-axis limits
plt.ylim([0, 3/2 * np.pi])

# Manually set the tick marks at certain values
plt.yticks([0, np.pi/4, np.pi/2, 3*np.pi/4, np.pi],
           ['0', r'$\frac{\pi}{4}$', r'$\frac{\pi}{2}$', r'$\frac{3\pi}{4}$', r'$\pi$'])

# Show the plot
plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

In [6]:
compare_2d(corrected_images[10:325], p, optimal_thetas)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (279, 1674) to (288, 1680) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
